In [21]:
import pandas as pd
import numpy as np
import pickle

In [2]:
test_df = pd.read_csv('./data/test.csv')
test_df

,index,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP
0,3220254,11100010,2054,11,3,2020-01-22 09:23:36,1.579685e+09
1,3220255,11100010,2054,9,4,NaN,NaN
2,3220256,11100010,2054,6,5,2020-01-22 09:27:26,1.579685e+09
3,3220257,11100010,2054,4,6,NaN,NaN
4,3220258,11100010,2054,147,7,NaN,NaN
...,...,...,...,...,...,...,...
1557459,4777713,12100232,740518021,657,9,2020-01-24 16:39:27,1.579884e+09
1557460,4777714,12100232,740518021,219,2,2020-01-24 17:28:00,1.579887e+09
1557461,4777715,12100232,740518021,221,3,NaN,NaN
1557462,4777716,12100232,740518021,242,4,NaN,NaN


In [3]:
route_index = 0
prev_seq    = 0
seq_data    = []
for idx, row in test_df.iterrows():
    if prev_seq == 0:
        prev_bus       = row['BUS_ID'     ]
        prev_seq       = row['BUSSTOP_SEQ']
        prev_route     = row['BUSROUTE_ID']
        prev_timestamp = row['TIMESTAMP'  ]
        min_timestamp  = row['TIMESTAMP'  ]
        cnt = 1
        continue
        
    if (row['BUSSTOP_SEQ']>prev_seq and row['BUS_ID'] == prev_bus and row['BUSROUTE_ID'] == prev_route):
        pass
    else:
        seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])
        min_timestamp = row['TIMESTAMP']
        route_index  +=1
        cnt           = 0
        
    prev_bus       = row['BUS_ID'     ]
    prev_seq       = row['BUSSTOP_SEQ']
    prev_route     = row['BUSROUTE_ID']
    prev_timestamp = row['TIMESTAMP'  ]
    test_df.at[idx, 'ROUTE_IDX'] = route_index
    cnt += 1
    
seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])

In [14]:
test_df

,index,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP,ROUTE_IDX
0,3220254,11100010,2054,11,3,2020-01-22 09:23:36,1.579685e+09,NaN
1,3220255,11100010,2054,9,4,NaN,NaN,0.0
2,3220256,11100010,2054,6,5,2020-01-22 09:27:26,1.579685e+09,0.0
3,3220257,11100010,2054,4,6,NaN,NaN,0.0
4,3220258,11100010,2054,147,7,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
1557459,4777713,12100232,740518021,657,9,2020-01-24 16:39:27,1.579884e+09,43975.0
1557460,4777714,12100232,740518021,219,2,2020-01-24 17:28:00,1.579887e+09,43976.0
1557461,4777715,12100232,740518021,221,3,NaN,NaN,43976.0
1557462,4777716,12100232,740518021,242,4,NaN,NaN,43976.0


In [18]:
cnt = 0

for idx, row in test_df.iterrows():
    try:
        test_df.at[idx, 'ROUTE_TIME'] = seq_data[int(row['ROUTE_IDX'])][3]
        test_df.at[idx, 'SEQ_NUM'   ] = seq_data[int(row['ROUTE_IDX'])][2]
    except:
        cnt = cnt+1

print("NANs", cnt)

NANs 1


In [20]:
test_df['DAY_OF_WEEK'     ] = pd.to_datetime(test_df['RECORD_DATE']).dt.dayofweek
test_df['HOUR_OF_DAY'     ] = pd.to_datetime(test_df['RECORD_DATE']).dt.hour
test_df['MINUTE_OF_HOUR'  ] = pd.to_datetime(test_df['RECORD_DATE']).dt.minute
test_df['SECOND_OF_MINUTE'] = pd.to_datetime(test_df['RECORD_DATE']).dt.second

In [22]:
bus_id_encoder     = pickle.load(open("./out/bus_id_encoder.pickle"    , "rb"))
route_id_encoder   = pickle.load(open("./out/route_id_encoder.pickle"  , "rb"))
busstop_id_encoder = pickle.load(open("./out/busstop_id_encoder.pickle", "rb"))

In [23]:
def timestamp_to_timeposition(time, starting_time = 21600): 
    if time < starting_time:
        return 0
    return (time - starting_time) % 86400

In [24]:
test_df['TIME_POSITION'] = test_df['TIMESTAMP'].apply(timestamp_to_timeposition)
test_df

,index,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE,TIME_POSITION
0,3220254,11100010,2054,11,3,2020-01-22 09:23:36,1.579685e+09,NaN,NaN,NaN,2.0,9.0,23.0,36.0,12216.0
1,3220255,11100010,2054,9,4,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN
2,3220256,11100010,2054,6,5,2020-01-22 09:27:26,1.579685e+09,0.0,1.488333,38.0,2.0,9.0,27.0,26.0,12446.0
3,3220257,11100010,2054,4,6,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN
4,3220258,11100010,2054,147,7,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557459,4777713,12100232,740518021,657,9,2020-01-24 16:39:27,1.579884e+09,43975.0,3.960833,8.0,4.0,16.0,39.0,27.0,38367.0
1557460,4777714,12100232,740518021,219,2,2020-01-24 17:28:00,1.579887e+09,43976.0,0.317778,4.0,4.0,17.0,28.0,0.0,41280.0
1557461,4777715,12100232,740518021,221,3,NaN,NaN,43976.0,0.317778,4.0,NaN,NaN,NaN,NaN,NaN
1557462,4777716,12100232,740518021,242,4,NaN,NaN,43976.0,0.317778,4.0,NaN,NaN,NaN,NaN,NaN


In [25]:
def busroute_position(busroute_id):
    return route_id_encoder.transform([int(busroute_id)])[0]

test_df['ROUTE_POSITION'] = test_df['BUSROUTE_ID'].apply(busroute_position)
test_df

,index,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE,TIME_POSITION,ROUTE_POSITION
0,3220254,11100010,2054,11,3,2020-01-22 09:23:36,1.579685e+09,NaN,NaN,NaN,2.0,9.0,23.0,36.0,12216.0,3
1,3220255,11100010,2054,9,4,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN,3
2,3220256,11100010,2054,6,5,2020-01-22 09:27:26,1.579685e+09,0.0,1.488333,38.0,2.0,9.0,27.0,26.0,12446.0,3
3,3220257,11100010,2054,4,6,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN,3
4,3220258,11100010,2054,147,7,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557459,4777713,12100232,740518021,657,9,2020-01-24 16:39:27,1.579884e+09,43975.0,3.960833,8.0,4.0,16.0,39.0,27.0,38367.0,352
1557460,4777714,12100232,740518021,219,2,2020-01-24 17:28:00,1.579887e+09,43976.0,0.317778,4.0,4.0,17.0,28.0,0.0,41280.0,352
1557461,4777715,12100232,740518021,221,3,NaN,NaN,43976.0,0.317778,4.0,NaN,NaN,NaN,NaN,NaN,352
1557462,4777716,12100232,740518021,242,4,NaN,NaN,43976.0,0.317778,4.0,NaN,NaN,NaN,NaN,NaN,352


In [29]:
def busstop_position(busstop_id, busstop_id_encoder=busstop_id_encoder):
    return busstop_id_encoder.transform([int(busstop_id)])[0]

test_df['BUSSTOP_POSITION'] = test_df['BUSSTOP_ID'].apply(busstop_position)
test_df

,index,BUSROUTE_ID,BUS_ID,BUSSTOP_ID,BUSSTOP_SEQ,RECORD_DATE,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE,TIME_POSITION,ROUTE_POSITION,BUSSTOP_POSITION
0,3220254,11100010,2054,11,3,2020-01-22 09:23:36,1.579685e+09,NaN,NaN,NaN,2.0,9.0,23.0,36.0,12216.0,3,10
1,3220255,11100010,2054,9,4,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN,3,8
2,3220256,11100010,2054,6,5,2020-01-22 09:27:26,1.579685e+09,0.0,1.488333,38.0,2.0,9.0,27.0,26.0,12446.0,3,5
3,3220257,11100010,2054,4,6,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN,3,3
4,3220258,11100010,2054,147,7,NaN,NaN,0.0,1.488333,38.0,NaN,NaN,NaN,NaN,NaN,3,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557459,4777713,12100232,740518021,657,9,2020-01-24 16:39:27,1.579884e+09,43975.0,3.960833,8.0,4.0,16.0,39.0,27.0,38367.0,352,648
1557460,4777714,12100232,740518021,219,2,2020-01-24 17:28:00,1.579887e+09,43976.0,0.317778,4.0,4.0,17.0,28.0,0.0,41280.0,352,218
1557461,4777715,12100232,740518021,221,3,NaN,NaN,43976.0,0.317778,4.0,NaN,NaN,NaN,NaN,NaN,352,220
1557462,4777716,12100232,740518021,242,4,NaN,NaN,43976.0,0.317778,4.0,NaN,NaN,NaN,NaN,NaN,352,241


In [33]:
train_df   = pd.read_csv("./out/cleaned_train.csv")
bus_routes = train_df['BUSROUTE_ID'].unique()

In [75]:
route_bus_dict = {}
for route_id in bus_routes:
    buses = np.unique(train_df[train_df['BUSROUTE_ID']==route_id]['BUS_ID']).astype(int)
    route_bus_dict[route_id] = buses
    #break

In [72]:
np.unique(train_df[train_df['BUSROUTE_ID']==251]).astype(int)

array([     0,      0,      0, ..., 477407, 490168, 849305])

In [80]:
import random
random.choice(route_bus_dict[3])

938

In [81]:
random.choice(list(route_bus_dict.keys()))
route_bus_dict.keys()

dict_keys([3, 5, 8, 12, 13, 16, 25, 30, 34, 39, 41, 45, 46, 49, 50, 55, 58, 59, 60, 61, 65, 66, 69, 70, 73, 74, 77, 80, 83, 89, 90, 92, 95, 96, 99, 100, 104, 107, 108, 114, 115, 117, 119, 121, 123, 127, 130, 133, 135, 137, 140, 142, 144, 147, 150, 152, 156, 160, 162, 164, 166, 168, 170, 173, 175, 179, 180, 188, 190, 193, 196, 198, 200, 201, 203, 206, 207, 209, 212, 214, 217, 219, 220, 221, 223, 224, 225, 232, 236, 240, 242, 246, 251, 253, 255, 259, 260, 265, 266, 268, 270, 271])

In [82]:
unique_buses = train_df['BUS_ID'].unique() 
unique_buses

array([  0,   1,   2, ...,  32, 217, 219], dtype=int64)

In [83]:
for idx, row in test_df.iterrows():
    bus_id   = int(row['BUS_ID'        ])
    route_id = int(row['ROUTE_POSITION'])
    try:
        bus_position = bus_id_encoder.transform([bus_id])[0]
    except:
        try:
            bus_position = random.choice(route_bus_dict[route_id])
        except:
            bus_position = random.choice(unique_buses)
            
    test_df.at[idx, 'BUS_POSITION'] = bus_position

In [84]:
test_df.to_csv('./out/cleaned_test.csv')

In [89]:
len(test_df['BUS_ID'])

1557464